# AirBNB data exploration

In this notebook, we're going to try to answer the following questions:

- What is affecting price variations?
- Does it cares that a host has several listings on price variations?
- When is a property on high or low price? (predictable)

In [1]:
from dask import dataframe as ddf
import dask
import pandas as pd
import numpy as np
dask.config.set(scheduler='processes')
#Custom function to download datasets from airbnb site.
from utils import download_datasets

In [6]:
#datasets to download
datasets_urls_boston = [
    'http://data.insideairbnb.com/united-states/ma/boston/2018-08-17/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/ma/boston/2018-08-17/data/calendar.csv.gz',
    'http://data.insideairbnb.com/united-states/ma/boston/2018-09-14/data/reviews.csv.gz'
]

datasets_urls_seattle = [
    'http://data.insideairbnb.com/united-states/wa/seattle/2018-08-16/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/wa/seattle/2018-08-16/data/calendar.csv.gz',
    'http://data.insideairbnb.com/united-states/wa/seattle/2018-09-14/data/reviews.csv.gz'
]
datasets_urls_austin = [
    'http://data.insideairbnb.com/united-states/tx/austin/2018-08-14/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/tx/austin/2018-08-14/data/calendar.csv.gz'
]


datasets_urls_losangeles = [
    'http://data.insideairbnb.com/united-states/ca/los-angeles/2018-09-08/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/ca/los-angeles/2018-09-08/data/calendar.csv.gz'
]

datasets_urls_neworleans = [
    'http://data.insideairbnb.com/united-states/la/new-orleans/2018-09-08/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/la/new-orleans/2018-09-08/data/calendar.csv.gz'
]


datasets_urls_sandiego = [
    'http://data.insideairbnb.com/united-states/ca/san-diego/2018-09-13/data/listings.csv.gz',
    'http://data.insideairbnb.com/united-states/ca/san-diego/2018-09-13/data/calendar.csv.gz'
]

download_datasets(datasets_urls_boston, 'boston_')
download_datasets(datasets_urls_seattle, 'seattle_')
download_datasets(datasets_urls_austin, 'austin_')
download_datasets(datasets_urls_losangeles, 'losangeles_')
download_datasets(datasets_urls_neworleans, 'neworleans_')
download_datasets(datasets_urls_sandiego, 'sandiego_')

datasets_cities = [
    'boston'
#     'seattle',
#     'austin',
#     'neworleans',
#     'sandiego',
]

File: boston_listings.csv.gz, in cache
File: boston_calendar.csv.gz, in cache
File: boston_reviews.csv.gz, in cache
File: seattle_listings.csv.gz, in cache
File: seattle_calendar.csv.gz, in cache
File: seattle_reviews.csv.gz, in cache
File: austin_listings.csv.gz, in cache
File: austin_calendar.csv.gz, in cache
File: losangeles_listings.csv.gz, in cache
File: losangeles_calendar.csv.gz, in cache
File: neworleans_listings.csv.gz, in cache
File: neworleans_calendar.csv.gz, in cache
File: sandiego_listings.csv.gz, in cache
File: sandiego_calendar.csv.gz, in cache


In [30]:
#Reading files and loading data into Dataframe
listings = [pd.read_csv(city + "_listings.csv.gz", low_memory=False, index_col='id') for city in datasets_cities]
listings = ddf.from_pandas(pd.concat(listings), npartitions=24)
calendar = [pd.read_csv(city + "_calendar.csv.gz", parse_dates=['date'], low_memory=False, index_col='listing_id') for city in datasets_cities]
calendar = ddf.from_pandas(pd.concat(calendar), npartitions=24)

/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [32]:
calendar.price = calendar.price.apply(lambda x: float(str(x).replace("$","").replace(",","")), meta=int)

In [34]:
calendar.available = calendar.available.apply(lambda x: True if x == 't' else False, meta=bool)